In [4]:
import os
import logging
import random
import gc
import time
import cv2
import math
import warnings
from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
import librosa

import tensorflow as tf
tf.config.optimizer.set_experimental_options({'layout_optimizer': False})
from tensorflow.keras import layers, models, applications, optimizers, losses, metrics, callbacks

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm

warnings.filterwarnings("ignore")
logging.basicConfig(level=logging.ERROR)

In [5]:
# Enable memory growth to prevent silent kernel crashes from OOM
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"Memory growth enabled for {len(gpus)} GPU(s).")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(f"Error setting memory growth: {e}")

# Verify setup
print("TensorFlow version:", tf.__version__)
print("Num GPUs Available: ", len(gpus))

Memory growth enabled for 1 GPU(s).
TensorFlow version: 2.19.0
Num GPUs Available:  1


In [6]:
class CFG:
    seed = 42
    debug = False
    print_freq = 100
    num_workers = 1

    # Set your project root directory
    PROJECT_ROOT_DIR = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))
 
    OUTPUT_DIR = os.path.join(PROJECT_ROOT_DIR, 'data/working/')
    
    train_datadir = os.path.join(PROJECT_ROOT_DIR, 'data/raw/train_audio')
    train_csv = os.path.join(PROJECT_ROOT_DIR, 'data/raw/train.csv')
    taxonomy_csv = os.path.join(PROJECT_ROOT_DIR, 'data/raw/taxonomy.csv')
    spectrogram_npy = os.path.join(PROJECT_ROOT_DIR, 'data/working/birdclef25-mel-spectrograms/birdclef2025_melspec_5sec_256_256.npy')
 
    model_name = 'EfficientNetB0'  
    
    # Keras Applications expect 3 input channels
    in_channels = 3 
    
    LOAD_DATA = True  
    FS = 32000
    TARGET_DURATION = 5.0
    TARGET_SHAPE = (256, 256)
    
    N_FFT = 1024
    HOP_LENGTH = 512
    N_MELS = 128
    FMIN = 50
    FMAX = 14000
    
    epochs = 10
    batch_size = 16
    
    n_fold = 5
    selected_folds = [0, 1, 2, 3, 4]   

    lr = 5e-4 
    weight_decay = 1e-5
    min_lr = 1e-6
    
    aug_prob = 0.5  
    mixup_alpha = 0.5
    
    def update_debug_settings(self):
        if self.debug:
            self.epochs = 2
            self.selected_folds = [0]

cfg = CFG()

In [7]:
def set_seed(seed=42):
    """
    Set seed for reproducibility
    """
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

set_seed(cfg.seed)

In [8]:
def audio2melspec(audio_data, cfg):
    """Convert audio data to mel spectrogram"""
    if np.isnan(audio_data).any():
        mean_signal = np.nanmean(audio_data)
        audio_data = np.nan_to_num(audio_data, nan=mean_signal)

    mel_spec = librosa.feature.melspectrogram(
        y=audio_data,
        sr=cfg.FS,
        n_fft=cfg.N_FFT,
        hop_length=cfg.HOP_LENGTH,
        n_mels=cfg.N_MELS,
        fmin=cfg.FMIN,
        fmax=cfg.FMAX,
        power=2.0
    )

    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
    mel_spec_norm = (mel_spec_db - mel_spec_db.min()) / (mel_spec_db.max() - mel_spec_db.min() + 1e-8)
    
    return mel_spec_norm

def process_audio_file(audio_path, cfg):
    """Process a single audio file to get the mel spectrogram"""
    try:
        audio_data, _ = librosa.load(audio_path, sr=cfg.FS)
        target_samples = int(cfg.TARGET_DURATION * cfg.FS)

        if len(audio_data) < target_samples:
            n_copy = math.ceil(target_samples / len(audio_data))
            if n_copy > 1:
                audio_data = np.concatenate([audio_data] * n_copy)

        start_idx = max(0, int(len(audio_data) / 2 - target_samples / 2))
        end_idx = min(len(audio_data), start_idx + target_samples)
        center_audio = audio_data[start_idx:end_idx]

        if len(center_audio) < target_samples:
            center_audio = np.pad(center_audio, 
                                 (0, target_samples - len(center_audio)), 
                                 mode='constant')

        mel_spec = audio2melspec(center_audio, cfg)
        
        if mel_spec.shape != cfg.TARGET_SHAPE:
            mel_spec = cv2.resize(mel_spec, cfg.TARGET_SHAPE, interpolation=cv2.INTER_LINEAR)

        # Replicate channel for Keras model
        mel_spec = np.stack([mel_spec, mel_spec, mel_spec], axis=-1)
        
        return mel_spec.astype(np.float32)
        
    except Exception as e:
        print(f"Error processing {audio_path}: {e}")
        return None

In [9]:
class SpecAugment(layers.Layer):
    def __init__(self, freq_mask_param, time_mask_param, num_freq_masks, num_time_masks, **kwargs):
        super(SpecAugment, self).__init__(**kwargs)
        self.freq_mask_param = freq_mask_param
        self.time_mask_param = time_mask_param
        self.num_freq_masks = num_freq_masks
        self.num_time_masks = num_time_masks

    def call(self, spec):
        sh = tf.shape(spec)
        B, H, W, C = sh[0], sh[1], sh[2], sh[3]

        # Frequency masking
        for _ in range(self.num_freq_masks):
            f = tf.random.uniform([], 0, self.freq_mask_param, dtype=tf.int32)
            f0 = tf.random.uniform([], 0, H - f, dtype=tf.int32)
            mask = tf.concat([
                tf.ones([B, f0, W, C]), 
                tf.zeros([B, f, W, C]), 
                tf.ones([B, H - f0 - f, W, C])
            ], axis=1)
            spec = spec * mask

        # Time masking
        for _ in range(self.num_time_masks):
            t = tf.random.uniform([], 0, self.time_mask_param, dtype=tf.int32)
            t0 = tf.random.uniform([], 0, W - t, dtype=tf.int32)
            
            # --- START: MODIFIED CODE ---
            mask = tf.concat([
                tf.ones([B, H, t0, C]), 
                tf.zeros([B, H, t, C]), 
                tf.ones([B, H, W - t0 - t, C]) # Corrected this line
            ], axis=2)
            # --- END: MODIFIED CODE ---
            spec = spec * mask
            
        return spec

# Data Generator
def data_generator(df, cfg, spectrograms, label_to_idx):
    for _, row in df.iterrows():
        samplename = row['samplename']
        spec = None

        if spectrograms and samplename in spectrograms:
            spec = spectrograms[samplename]
            # Keras Applications expect 3 channels
            spec = np.stack([spec, spec, spec], axis=-1) 
        elif not cfg.LOAD_DATA:
            spec = process_audio_file(row['filepath'], cfg)

        if spec is None:
            spec = np.zeros((*cfg.TARGET_SHAPE, cfg.in_channels), dtype=np.float32)

        # Create one-hot encoded label
        target = np.zeros(len(label_to_idx), dtype=np.float32)
        target[label_to_idx[row['primary_label']]] = 1.0
        
        if isinstance(row['secondary_labels'], str):
            secondary_labels = eval(row['secondary_labels'])
            for label in secondary_labels:
                if label in label_to_idx:
                    target[label_to_idx[label]] = 1.0
                    
        yield spec, target

# MixUp Function for tf.data
def mixup(ds_one, ds_two, alpha):
    ds_one = ds_one.map(lambda x, y: (x, y, 1.))
    ds_two = ds_two.map(lambda x, y: (x, y, 0.))
    
    ds = tf.data.Dataset.zip((ds_one, ds_two))
    
    # Corrected function definition below
    def do_mixup(one, two):
        (img1, label1, _), (img2, label2, _) = one, two
        
        # Unpack the images and apply mixup
        lam = tf.random.uniform([], 0, alpha)
        
        img = lam * img1 + (1. - lam) * img2
        label = lam * label1 + (1. - lam) * label2
        return img, label

    return ds.map(do_mixup, num_parallel_calls=tf.data.AUTOTUNE)

In [10]:
def get_model(cfg):
    
    input_shape = (*cfg.TARGET_SHAPE, cfg.in_channels)
    
    # Input layer
    inputs = layers.Input(shape=input_shape)
    
    # Augmentation Layer
    if cfg.aug_prob > 0:
        x = layers.RandomFlip("horizontal")(inputs)
        # Apply SpecAugment with a certain probability
        if tf.random.uniform([]) < cfg.aug_prob:
             x = SpecAugment(freq_mask_param=25, time_mask_param=40, num_freq_masks=2, num_time_masks=2)(x)
    else:
        x = inputs
    
    # Backbone
    backbone = applications.EfficientNetB0(
        include_top=False,
        weights='imagenet',
        input_shape=input_shape
    )
    
    # EfficientNetB0 is not trainable by default when loaded this way
    backbone.trainable = True
    
    x = backbone(x)
    
    # Classifier head
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.2)(x)
    
    # Output layer for multi-label classification
    outputs = layers.Dense(cfg.num_classes, activation='sigmoid')(x)
    
    model = models.Model(inputs, outputs)
    
    return model

In [11]:
def run_training(df, cfg):
    """Training function using Keras/TensorFlow"""

    taxonomy_df = pd.read_csv(cfg.taxonomy_csv)
    species_ids = taxonomy_df['primary_label'].tolist()
    cfg.num_classes = len(species_ids)
    label_to_idx = {label: idx for idx, label in enumerate(species_ids)}

    if cfg.debug:
        cfg.update_debug_settings()
        df = df.sample(min(2000, len(df)), random_state=cfg.seed).reset_index(drop=True)

    spectrograms = None
    if cfg.LOAD_DATA:
        print("Loading pre-computed mel spectrograms from NPY file...")
        try:
            spectrograms = np.load(cfg.spectrogram_npy, allow_pickle=True).item()
            print(f"Loaded {len(spectrograms)} pre-computed mel spectrograms")
        except Exception as e:
            print(f"Error loading pre-computed spectrograms: {e}. Will generate on-the-fly.")
            cfg.LOAD_DATA = False
    
    if not cfg.LOAD_DATA:
        df['filepath'] = cfg.train_datadir + '/' + df.filename
    df['samplename'] = df.filename.map(lambda x: x.split('/')[0] + '-' + x.split('/')[-1].split('.')[0])
        
    skf = StratifiedKFold(n_splits=cfg.n_fold, shuffle=True, random_state=cfg.seed)
    
    best_scores = []
    
    for fold, (train_idx, val_idx) in enumerate(skf.split(df, df['primary_label'])):
        if fold not in cfg.selected_folds:
            continue
            
        print(f'\n{"="*30} Fold {fold} {"="*30}')
        
        train_df = df.iloc[train_idx].reset_index(drop=True)
        val_df = df.iloc[val_idx].reset_index(drop=True)
        
        # Create tf.data datasets
        train_ds = tf.data.Dataset.from_generator(
            lambda: data_generator(train_df, cfg, spectrograms, label_to_idx),
            output_signature=(
                tf.TensorSpec(shape=(*cfg.TARGET_SHAPE, cfg.in_channels), dtype=tf.float32),
                tf.TensorSpec(shape=(cfg.num_classes,), dtype=tf.float32)
            )
        )
        
        val_ds = tf.data.Dataset.from_generator(
            lambda: data_generator(val_df, cfg, spectrograms, label_to_idx),
            output_signature=(
                tf.TensorSpec(shape=(*cfg.TARGET_SHAPE, cfg.in_channels), dtype=tf.float32),
                tf.TensorSpec(shape=(cfg.num_classes,), dtype=tf.float32)
            )
        )
        
        # Apply MixUp
        if cfg.mixup_alpha > 0:
            train_ds_one = train_ds.shuffle(buffer_size=len(train_df))
            train_ds_two = train_ds.shuffle(buffer_size=len(train_df))
            train_ds = mixup(train_ds_one, train_ds_two, alpha=cfg.mixup_alpha)

        # Batch and prefetch
        train_ds = train_ds.batch(cfg.batch_size).prefetch(tf.data.AUTOTUNE)
        val_ds = val_ds.batch(cfg.batch_size).prefetch(tf.data.AUTOTUNE)

        # Model, Optimizer, Loss, and Metrics
        model = get_model(cfg)
        
        # Manually calculate the number of steps per epoch
        steps_per_epoch = len(train_df) // cfg.batch_size
        
        lr_schedule = optimizers.schedules.CosineDecay(
            cfg.lr,
            decay_steps=steps_per_epoch * cfg.epochs, # Use calculated steps
            alpha=cfg.min_lr / cfg.lr
        )
        
        optimizer = optimizers.AdamW(learning_rate=lr_schedule, weight_decay=cfg.weight_decay)
        
        loss = losses.BinaryCrossentropy() # Sigmoid activation is in the model
        
        auc_metric = metrics.AUC(multi_label=True, name='auc', from_logits=False)
        
        model.compile(optimizer=optimizer, loss=loss, metrics=[auc_metric])

        # Callbacks
        model_checkpoint = callbacks.ModelCheckpoint(
            f"model_fold{fold}.keras", 
            monitor='val_auc', 
            mode='max', 
            save_best_only=True,
            verbose=1
        )
        
        history = model.fit(
            train_ds,
            epochs=cfg.epochs,
            validation_data=val_ds,
            callbacks=[model_checkpoint]
        )
        
        best_fold_auc = max(history.history['val_auc'])
        best_scores.append(best_fold_auc)
        print(f"\nBest AUC for fold {fold}: {best_fold_auc:.4f}")
        
        # Clear memory
        del model, train_ds, val_ds
        tf.keras.backend.clear_session()
        gc.collect()
    
    print("\n" + "="*60)
    print("Cross-Validation Results:")
    for i, score in enumerate(best_scores):
        print(f"Fold {cfg.selected_folds[i]}: {score:.4f}")
    print(f"Mean AUC: {np.mean(best_scores):.4f}")
    print("="*60)

In [12]:
if __name__ == "__main__":
    print("\nLoading training data...")
    train_df = pd.read_csv(cfg.train_csv)

    print("\nStarting training...")
    run_training(train_df, cfg)
    
    print("\nTraining complete!")


Loading training data...

Starting training...
Loading pre-computed mel spectrograms from NPY file...
Loaded 28564 pre-computed mel spectrograms

============================== Fold 0 ==============================


I0000 00:00:1753645007.415090  152033 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22293 MB memory:  -> device: 0, name: NVIDIA A30, pci bus id: 0000:50:00.0, compute capability: 8.0


Epoch 1/10


2025-07-27 12:37:24.575381: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] ShuffleDatasetV3:4: Filling up shuffle buffer (this may take a while): 16217 of 22851
2025-07-27 12:37:29.803747: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:482] Shuffle buffer filled.
2025-07-27 12:37:39.804944: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] ShuffleDatasetV3:5: Filling up shuffle buffer (this may take a while): 13112 of 22851
2025-07-27 12:37:47.758838: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:482] Shuffle buffer filled.
I0000 00:00:1753645067.784464  152675 service.cc:152] XLA service 0x7f9da4015cf0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753645067.784492  152675 service.cc:160]   StreamExecutor device (0): NVIDIA A30, Compute Capability 8.0
2025-07-27 12:37:49.296612: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH

      1/Unknown 88s 88s/step - auc: 0.0745 - loss: 0.7062

...

1429/1429 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - auc: 0.7341 - loss: 0.0174
Epoch 10: val_auc improved from 0.79591 to 0.84578, saving model to model_fold1.keras


2025-07-27 13:06:17.892125: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 13482737626494985881
2025-07-27 13:06:17.892193: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 9976557840587246764


1429/1429 ━━━━━━━━━━━━━━━━━━━━ 73s 33ms/step - auc: 0.7341 - loss: 0.0174 - val_auc: 0.8458 - val_loss: 0.0136

Best AUC for fold 1: 0.8458

============================== Fold 2 ==============================
Epoch 1/10


2025-07-27 13:06:55.238033: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] ShuffleDatasetV3:96: Filling up shuffle buffer (this may take a while): 17400 of 22851
2025-07-27 13:06:59.171477: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:482] Shuffle buffer filled.
2025-07-27 13:07:09.173669: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] ShuffleDatasetV3:97: Filling up shuffle buffer (this may take a while): 14571 of 22851
2025-07-27 13:07:14.684417: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:482] Shuffle buffer filled.
2025-07-27 13:07:17.936121: W tensorflow/compiler/tf2xla/kernels/random_ops.cc:108] Warning: Using tf.random.uniform with XLA compilation will ignore seeds; consider using tf.random.stateless_uniform instead if reproducible behavior is desired. functional_1/spec_augment_1/random_uniform
2025-07-27 13:07:18.062195: W tensorflow/core/framework/op_kernel.cc:1857] OP_REQUIRES failed at fill_op.cc:55 : INVALID_ARGUMENT: Input 0 to node `

InvalidArgumentError: Graph execution error:

Detected at node functional_1/spec_augment_1/ones defined at (most recent call last):
<stack traces unavailable>
Input 0 to node `functional_1/spec_augment_1/ones` with op Fill must be a compile-time constant.

XLA compilation requires that operator arguments that represent shapes or dimensions be evaluated to concrete values at compile time. This error means that a shape or dimension argument could not be evaluated at compile time, usually because the value of the argument depends on a parameter to the computation, on a variable, or on a stateful operation such as a random number generator.

Stack trace for op definition: 
File "data/homezvol2/ddlin/.local/share/uv/python/cpython-3.10.2-linux-x86_64-gnu/lib/python3.10/runpy.py", line 196, in _run_module_as_main
File "data/homezvol2/ddlin/.local/share/uv/python/cpython-3.10.2-linux-x86_64-gnu/lib/python3.10/runpy.py", line 86, in _run_code
File "share/crsp/lab/pkaiser/ddlin/mids/DATASCI207_

In [ ]:
# This notebooks needs 64GB of mem to run